In [3]:
import glob
import json
import re
import requests
from bs4 import BeautifulSoup

In [4]:
def readData(fileName):
    try:
        with open('./etwarm_json/' + fileName, "rt", encoding="utf-8") as f:
            data = json.load(f)
            return data
    except FileNotFoundError as e:
        print(e)

In [5]:
def saveJson(data, filename):
    with open(filename, "w", encoding="utf8") as f:
        json.dump(data, f, ensure_ascii=False)

In [6]:
# File list
etwarm = glob.glob('./etwarm_json/*.json')

# Extend file names to List
etwarm0 = re.findall('\.\/etwarm_json\\\\(\w+\.json)', etwarm[0])
for i in range(1, len(etwarm)):
    etwarm0.extend(re.findall('\.\/etwarm_json\\\\(\w+\.json)', etwarm[i]))

In [7]:
etwarm0

['178343.json',
 '178378.json',
 '178450.json',
 '178501.json',
 '178707.json',
 '179181.json',
 '179257.json',
 '179368.json',
 '179476.json',
 '179529.json',
 '179587.json',
 '179730.json',
 '179791.json',
 '179908.json',
 '179952.json',
 '180135.json',
 '180273.json',
 '180510.json',
 '180534.json',
 '180535.json',
 '180564.json',
 '180607.json',
 '180635.json',
 '180762.json',
 '180765.json',
 '180778.json',
 '180916.json',
 '180984.json',
 '180985.json',
 '181055.json',
 '181116.json',
 '181139.json',
 '181144.json']

In [8]:
len(etwarm0)

33

In [115]:
data = readData(etwarm0[3])
soup = BeautifulSoup(data["html"], "lxml")

In [116]:
soup

<html><body><p>網站維護中</p></body></html>

***

* id

* cityID

In [117]:
postalCode = {
    "中正區": 100, "大同區": 103, "中山區": 104, "松山區": 105, "大安區": 106, "萬華區": 108, "信義區": 110,
    "士林區": 111, "北投區": 112, "內湖區": 114, "南港區": 115, "文山區": 116, "萬里區": 207, "金山區": 208,
    "板橋區": 220, "汐止區": 221, "深坑區": 222, "石碇區": 223, "瑞芳區": 224, "平溪區": 226, "雙溪區": 227,
    "貢寮區": 228, "新店區": 231, "坪林區": 232, "烏來區": 233, "永和區": 234, "中和區": 235, "土城區": 236,
    "三峽區": 237, "樹林區": 238, "鶯歌區": 239, "三重區": 241, "新莊區": 242, "泰山區": 243, "林口區": 244,
    "蘆洲區": 247, "五股區": 248, "八里區": 249, "淡水區": 251, "三芝區": 252, "石門區": 253
}

In [118]:
# Use Google API to get postal code, lat, lng
api_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key="

def getLocation(address):
    url = api_url.format(address)
    res = requests.get(url)
    location_dict = res.json()
    cityID = location_dict['results'][0]['address_components'][-1]['short_name']
    lat = location_dict['results'][0]['geometry']['location']['lat']
    lng = location_dict['results'][0]['geometry']['location']['lng']
    res.close()
    return cityID, lat, lng

In [119]:
soup.select('div[class="obj_data_route"]')[0].text

IndexError: list index out of range

In [120]:
city = soup.select('div[class="obj_data_route"]')[0].select('a')[2].text.lstrip().rstrip()

IndexError: list index out of range

In [75]:
postalCode[city]

241

* url

In [76]:
data["url"]

'http://www.etwarm.com.tw/rent-178343'

* title

In [77]:
soup.select('div[class="obj_data_basic"]')[0].select('h1')[0].text

'新北三重臨捷運站單人套房出租'

* address

In [78]:
data_basic = soup.select('ul[class="data_basic"]')[0].select('li')

In [79]:
address = str(re.findall('<li>地址 : (\w+)<\/li>', str(data_basic))).lstrip("\'\[").rstrip("\'\]")

In [80]:
address

'新北市三重區介壽路39巷'

* pattern (水泥、木板)

In [81]:
"水泥"

'水泥'

* floor

In [82]:
str(data_basic)

'[<li>\n<div class="obj_data_contain fl">\n<span class="price">6,000</span> 元 / 月\n            </div>\n</li>, <li>面積 : <span class="space">7坪</span></li>, <li>用途/型態 : 獨立套房 / 公寓</li>, <li>樓層 : 3/5(總樓層)</li>, <li>地址 : 新北市三重區介壽路39巷</li>, <li>\n<a href="rent/house_warning.html" id="etwarm_rentwarning">\n<img height="20" src="images/icon_Warning.png" style="vertical-align: middle;" width="20"/>\n<span style="color: #ff0000">租屋注意事項&amp;房屋租賃定型化契約</span>\n</a>\n</li>]'

In [83]:
int(''.join(re.findall('<li>樓層 : (\d+)\/\d+\(總樓層\)</li>', str(data_basic))))

3

* stories

In [84]:
int(''.join(re.findall('<li>樓層 : \d+\/(\d+)\(總樓層\)</li>', str(data_basic))))

5

* label (套、雅、住)

In [85]:
''.join(re.findall('<li>用途/型態 : \w*(套|雅)', str(data_basic)))

'套'

* rent

In [86]:
soup.select('div[class="obj_data_contain fl"]')

[<div class="obj_data_contain fl">
 <span class="price">6,000</span> 元 / 月
             </div>]

In [87]:
soup.select('div[class="obj_data_contain fl"]')[0]

<div class="obj_data_contain fl">
<span class="price">6,000</span> 元 / 月
            </div>

In [88]:
int(soup.select('div[class="obj_data_contain fl"]')[0].select('span')[0].text.replace(',', ''))

6000

* cityID, lat, lng

In [89]:
cityID, lat, lng = getLocation(address)

In [90]:
cityID

'241'

In [91]:
lat

25.0704884

In [92]:
lng

121.5011932

* space

In [93]:
data_basic = soup.select('ul[class="data_basic"]')[0].findAll('li')
str(data_basic)

'[<li>\n<div class="obj_data_contain fl">\n<span class="price">6,000</span> 元 / 月\n            </div>\n</li>, <li>面積 : <span class="space">7坪</span></li>, <li>用途/型態 : 獨立套房 / 公寓</li>, <li>樓層 : 3/5(總樓層)</li>, <li>地址 : 新北市三重區介壽路39巷</li>, <li>\n<a href="rent/house_warning.html" id="etwarm_rentwarning">\n<img height="20" src="images/icon_Warning.png" style="vertical-align: middle;" width="20"/>\n<span style="color: #ff0000">租屋注意事項&amp;房屋租賃定型化契約</span>\n</a>\n</li>]'

In [94]:
data_basic = soup.select('ul[class="data_basic"]')[0].findAll('li')
float(''.join(re.findall('<span class="space">(\w+)坪</span>', str(data_basic))))

7.0

* update

In [95]:
data["update"]

'2017-08-16'

* landlord

In [96]:
soup.select('div[class="data_store"]')[0].text.split()

['屋主林先生', '0988370162', '(屋主聲明：仲介勿擾)']

In [97]:
landlord_name = soup.select('div[class="data_store"]')[0].text.split()[0]

In [98]:
landlord_name = re.findall('!?屋!?主(.+)', landlord_name)

In [99]:
landlord_name = str(landlord_name).lstrip("\'\[").rstrip("\'\]")

In [100]:
landlord_name

'林先生'

In [101]:
landlord_phone = soup.select('div[class="data_store"]')[0].text.split()[1]

In [102]:
landlord = landlord_name + "," + landlord_phone

In [103]:
landlord

'林先生,0988370162'

In [104]:
landlord_re = str(soup.select('div[class="data_store"]')[0].text.split())

In [105]:
str(''.join(re.findall('\d{10}', landlord_re)))

'0988370162'

* description

In [106]:
description = lambda x: x if soup.select('section[id="obj_characteristic"]')[0].text else "NA"

In [107]:
description

<function __main__.<lambda>>

In [108]:
try:
    description = ''.join(soup.select('section[id="obj_characteristic"]')[0].text.split())
except:
    "NA"

In [114]:
description

'特色推薦(新北市三重區介壽路39巷..歡迎先去看街道環境.有意者再來電)(.租金6000元含水費、寬頻網路、第四台)(鄰近光榮國中.捷運三重國小站)巷子小，有對外窗及曬衣空間獨立衛浴與門戶.水泥隔間套房約6坪大.為狹長空間..(扣除家具家電後.只有能再容納約一個單人床大小的走道.若不嫌棄者歡迎)交通便利近蘆洲線三重國小捷運站.台北橋等生活機能優近三和夜市商圈.**絕對謝絕養寵物的房客**聯絡電話:0988370162林先生立即可入住提供設備：床.衣櫃.電視.冷氣.冰箱.熱水器.洗衣機.有線電視.網路生活機能：近便利商店；傳統市場；學校；夜市;捷運站;可遷入搶拼出租！！★本處套房皆為房東自租，絕非仲介或二房東，看屋不論是否承租，絕對不收取任何服務或仲介費用，歡迎來電預約看屋！！水費、寬頻網路、第四台.每月至少省1０００元，不景氣中房東咬牙苦撐'